# 최적 제어 아이디어

## 목적함수 정의
우리의 목표는 에너지를 절감하면서 세 실내기의 온도가 균형을 맞춰서 고루 떨어지고
최대한 빨리 목표 온도에 도달하게 하는것

### 기본 목적 함수
$$
J = w_1 \cdot \text{Var}(t_1, t_2, t_3) + w_2 \cdot \max(t_1, t_2, t_3) + w_3 \cdot E_{penalty}
$$

### 에너지 페널티 근사식
$$
E_{penalty} \approx \sum_{i=1}^{3} \left( w_F \cdot F_i + w_T \cdot (T_{bdy, i} - T_{set, i}) \right)
$$ 


각 실내기들의 상태 및 공통상태 액션을 인풋으로 받아서 각 실내기의 도달 시간을 예측하는 모델이 있다고 가정한다


실제 제어에서 실내기들의 상태와 공통 상태는 센서 데이터로 주어진다.

또한 가능한 액션 조합은 정해져있다. 
Tset을 2도로 binning
```
    풍량 1, 2, 3, 4 (4) 
    온도는 16, 18, 20, 22, 24 (5) 
    ------------------------------(4*5 = 20)
    OFF의 경우의 수 하나 (1) 
    ------------------------------(20+1 = 21)
    각 실내기마다니까.
    ------------------------------(21^3 = 9,261)
```

현재 관측된 s와 가능한 액션 조합에 대해서 모델을 이용해서 도달 시간을 구한다

-> 목적함수를 계산한다

-> 목적함수를 최소화하는 액션을 선택한다

-> 일정 주기(ex 5분)로 반복한다. 


### 최종 목적 함수 (부하 보상 포함)
$$
J_{final} = J - w_4 \sum_{i=1}^{3} \left( \frac{\tau_{real, i}}{\tau_{model, i}} - 1 \right) u_i
$$





모델의 시정수
$$
T(\hat{t}) = T_{goal} = T_{eq} + (T_0 - T_{eq}) e^{-\frac{\hat{t}}{\tau_{model}}}
$$

$$
% 시정수(tau)에 대해 정리한 수식
\tau_{model} = \frac{-\hat{t}}{\ln\left(\frac{T_{goal} - T_{eq}}{T_0 - T_{eq}}\right)}
$$

실측된 실제 시정수 
$$
T(n) = T_{eq} + (T_0 - T_{eq}) e^{-\frac{n}{\tau_{real}}}
$$

% 실제 시정수(tau_real) 역산 공식
$$
\tau_{real} = \frac{n}{\ln\left(\frac{T_0 - T_{eq}}{T(n) - T_{eq}}\right)}
$$ 


제어 입력 강도를 스칼라화 하기위한 u 정의
$$
u = \alpha \frac{F - F_{\min}}{F_{\max} - F_{\min}} + (1-\alpha) \frac{T_{set, \max} - T_{set}}{T_{set, \max} - T_{set, \min}}
$$


만약 부하가 크면 $\frac{\tau_{real, i}}{\tau_{model, i}}$ 은 1보다 크고 부하 보정 항은 음수가 된다 - > u를 크게 만들려고 한다

만약 부하가 없으면  $\frac{\tau_{real, i}}{\tau_{model, i}}$는 1이되고 부하 보정항은 0이되어 원래 목적함수를 따르게 된다

예측된 것보다 실제가 빠르게 떨어지면 $\frac{\tau_{real, i}}{\tau_{model, i}}$ 는 1보다 작게 되고 부하 보정항은 양수가 되어 u를 최소화하려고 한다